## Generics
Generics enables type to be parameters. In doing so it provides the following benefits:
- Code re-use

In [ ]:
public int add(int a, int b){
    return a + b;
}

public float add(float a, float b){
    return a + b;
}

    Can be written as:

In [ ]:
public T add(T a, T b){ // But we would need to limit types T can be
    return a + b;       
}

- Elimination of casts

In [ ]:
List oldList = new ArrayList();
oldList.add("some text");
String str = (String) oldList.get(0);

// Can be written as
List<String> newList = new ArrayList<>();
newList.add("some text");
String str = newList.get(0);

This introduces type safety, saving us from class cast exceptions.

## Generic Class
We define a generic class by specifying a type parameter inside <>. The type parameter need not be single letter, it can be a full word. Type parameter naming convention:
- `E`: element
- `K`: key
- `V`: value
- `T`: type

In [ ]:
class Box<T> { // Specifying T here means this type parameter is now available
               // throughout the class
    private T t;

    public Box(T t){
        this.t = t;
    }

    public T get() {
        return t;
    }

    public void set(T t) {
        this.t = t;
    }
}

Box<Number> numBox = new Box<>(5); // Number is the type argument
                                   // Type inference allows us to omit the type in the second <>

Multiple type parameters can also be specified 

In [ ]:
class Pair<K, V> { // Different type parameters are just separated by a comma
    private K key;
    private V value;

    public void set(K key, V value) {
        this.key = key;
        this.value = value;
    }

    public V getValue() {
        return this.value;
    }

    public V getKey() {
        return this.value;
    }
}

A type parameter can also be substituted with another parameterised type:

In [ ]:
OrderedPair<String, Box<Integer>> pair;

We can also have a generic method inside a non-generic class

In [ ]:
class Util {
    // Notice <T>, we have to define type parameter first
    // since class is non-generic
    public static <T> void fromArrayToCollection(T[] a, Collection<T> c) { // Type parameter T is limited
                                                                           // to this method
        for (T o : a) {
            c.add(o);
        }
    }
}

Util.<String>fromArrayToCollection(strArray, strCollection); // Typical usage, though we can ususally remove the type argument
Util.fromArrayToCollection(strArray, strCollection) // works

**Raw type:** is the name of a generic class or interface without any type arguments. Mixing raw and generic type results in compiler warnings. Example:

In [ ]:
Box box = new Box();

// If we assign a raw type to a parameterized type, you get a warning
Box<String> stringBox = new Box("Hello");

// Since the reference is generic, we can only call
stringBox.set("Another value");
stringBox.set(55);    // Error

// The below statement is also allowed for backword compatibility:
Box rawBox = new Box<Integer>(5);
rawBox.set("Greetings); // Works since reference is raw

Example showing how mixing raw and parameterised types can lead to error:

In [ ]:
Box<String> stringBox = new Box(1);
String boxed = stringBox.get(); // ClassCastException

Or

In [ ]:
Box stringBox = new Box<Integer>(1);
String boxed = (String) stringBox.get(); // ClassCastException

**Bounded Type Parameter:** lets us restrict the types that can be used as type arguments.

In [ ]:
class Complex <T extends Number> {
    public T real;
    public T imaginary;
}

In [ ]:
// Multiple bounds, useful if T implements multiple interfaces
class AnimalCollection <T extends Vertebrate & Warmblooded> { // The first type parameter has to be class
    // ...
}

class Mammal extends Vertebrate implements Warmblooded { // Mammal can be used as Type argument
    // ...
}

class Reptile extends Vertebrate{ // Reptiles can't be used as Type argument
    // ...
}

## Generics And Inheritance
Examples below illustrate an is-a relationship indicating parent-child inheritance:  
> An `Integer` *is an* `Object`  
> A `Double` *is a* `Number`  

However,
> A `Box<Integer>` *is not a* `Box<Number>`  
> A `Box<Integer>` *is an* `Object`  

In general, given two concrete types *A* and *B*, `MyClass<A>` has no relationship with `MyClass<B>` regardless of whether *A* and *B* are related or not.  

The following relationships are true:
> A `List<Integer>` *is a* `Collection<Integer>`  
> An `ArrayList<Float>` *is a* `List<Float>`  
> An `ArrayList<Float>` *is not a* `List<Number>`  

Sybtype relationship is preserved as long as type argument is not varied. Let's consider the below example:

In [ ]:
class NamedList<N, E> extends List<E> {
    // ...
}

For the above definition:

> A `NamedList<String, String>` *is a* `List<String>`  
> A `NamedList<Integer, String>` *is a* `List<String>`  
> A `NamedList<String, Number>` *is not a* `List<String>`  

As a result of above statements, the following substitutions are not allowed:

In [ ]:
public void compute(Box<Number> input){
    // ...
}

// Below method call is invalid
compute(new Box<Integer>()); // Error

// Passing a raw type is legal, but we get warning
compute(new Box());

But why is `List<Integer>` not a subtype of `List<Number>`? The below code illustrates the reasoning:

In [ ]:
List<Integer> integers = new ArrayList<>();
integers.add(1); integers.add(2); integers.add(3);

List<Number> numbers = integers;
numbers.add(4.5); // We have allowed a double to be put inside a list of integers

If we want to extend a generic class, the child class must include the type specified by the parent class.

In [ ]:
// Valid
class SpecialList<T> extends ArrayList<T> {

}

// Valid, child class introducing its own parameter
class SpecialList<T, U> extends ArrayList<T> {

}

// Valid, generic class can extend non-generic class
class SpecialList<T> extends ArrayList {

}

// Error, child class must have all type parameters
// of the parent class
class SpecialList<T> extends ArrayList<U> {

}

// Error, child class should also have the interface's type
// parameter. Type parameters of child class should be union 
// of type parameters of generic class and generic interface
class SpecialList<T> extends ArrayList<T> implements Comparable<U>{
    
}

// Error, non-generic class can’t extend generic class
class SpecialList extends ArrayList<T>{
    
}

// But this is fine, since we are using a pre-defined class
class SpecialStringList extends ArrayList<String>{
    
}

// String is a type parameter, not String class! (For both LHS and RHS)
class SpecialStringList<String> extends ArrayList<String>{
    
}
SpecialStringList<Integer> integerList = new SpecialStringList<>();

When we extend a generic class with bounded type parameter, type parameter must be replaced by either upper bound or it’s sub classes.

In [ ]:
class Parent<T extends Number> {

}

// T can be any type
class Child<T> extends Parent<Integer> {

}

## Type Inference
Allows us to invoke a generic method as we would an ordinary method:

In [ ]:
public static <U> void addBox(U item, List<Box<U>> boxes) {
    Box<U> box = new Box<>();
    box.set(item);

    boxes.add(box);
)

// Above method can be called with or without specifying the type argument
// Java compiler figures out the type from the type of method arguments
<Integer>addBox(Integer.valueOf(10), new ArrayList<Box<Integer>>());
addBox(Integer.valueOf(10), new ArrayList<>()); // equivalent to above

Take another example below (wildcards covered later):

In [ ]:
class Copy {
    public static <E> void copy(List<? super E> destination, List<? extends E> source) {
        for (E e : source) {
            destination.add(e);
        }
    }
}

// Numbers
List<Number> numbers = new ArrayList<>();
numbers.add(0.0);
// Integers
List<Integer> integers = new ArrayList<>();
integers.add(1); integers.add(2);
// Copy integers into numbers
Copy.copy(numbers, integers);  // Type is inferred

In the above example, there are 3 possibilities:

In [ ]:
Copy.copy(numbers, integers);          // implicit parameter, inferred as Integer
Copy.<Number>copy(numbers, integers);  // numbers array is ok for ? super Number and integers is ok for ? extends Number
Copy.<Integer>copy(numbers, integers); // numbers array is ok for ? super Integer and integers is ok for ? extends Integer

## Wildcards
There may be times when we want to restrict the kinds of types that are allowed to be passed to a type parameter. For example, say you want to write a method that works on `List<Integer>`, `List<Double>`, and `List<Number>`. We use **upper bounded wildcard**:

In [ ]:
publi double listSummation(List<? extends Number> list) {
    double s = 0.0;
    for(Number n: list) {
        s += n.doubleValue();
    }

    return s;
}

Another example:

In [ ]:
public void process(List<? extends Number> src, List<? extends Number> dest) {
    // src and destination can be List<Integer> and List<Double> respectively
}

Note that this is different from:

In [ ]:
public <T extends Number> void process(List<T> src, List<T> dest) {
    // src and destination both need to be the same type
}

But, the following code has the same functionality as the wildcard version

In [ ]:
public <T extends Number, U extends Number> void process(List<T> src, List<U> dest) {
    // src and destination can be List<Integer> and List<Double> respectively
}

The wildcard version however is more concise and readable.  

There is also **lower bounded wildcard**:

In [ ]:
public void process(List<? super Number> src) {
    // Type parameter of List must atleast be Number
}

public static void add10Numbers(List<? super Integer> list) {
    for(int i=1; i<=10; i++) {
        list.add(i);        
    }    
}

And **unbounded wildcard**. Expression `List<?>` represents a list of unknown type. This particular variation is used when the code is only using methods that do not depend upon the type parameter. In case of `List`, that would be `List.size()` or `List.clear()`. This is also the reason why `Class<?>` is so prevalent, we normally don't utilize the methods of the individual class in these scenarios.  

Note that `List<?>` is different from `List<Object>`, therefore:

In [ ]:
void printObjectList(List<Object> list) {  // can only print list of Objects
    // ...
}

void printAnyList(List<?> list) {  // can print list of anything
    // ...
}

So when to use `extends` and when to use `super`? Follow the GET and PUT principle:
- Getting something, extracting values (GET): use `extends`
- Putting or inserting some values (PUT): use `super`
- Don't use wildcard when you GET and PUT both

In [ ]:
// Getting numbers out of List, use extends
public static double sum(List<? extends Number> numbers) {
    double sum = 0.0;
    for(Number n: numbers) {
        sum += n.doubleValue();
    }
    return sum;
}

// All these arrays are allowed for the above call, not possible if extends was not used
// - Arrays.asList(1,2,3);
// - Arrays.asList(1.0,2.0,3.0);
// - Arrays.asList(1,2.0,3);

In [ ]:
// Filling a list with integers, use super
public static void fill(List<? super Integer> list, int number) {
    for(int i=0; i<number; i++) {
        list.add(i);
    }
}

// All these arrays are allowed for the above call, not possible if super was not used
// - Arrays.asList(1,2,3);     -- Integer
// - Arrays.asList(1,2.0,3.0); -- Number
// - Arrays.asList(1,2.0,"3"); -- Object

In [ ]:
// Get and put both on the same list, don't use wildcard
public static double fillAndSum(List<Number> list) {
    fill(list, 10);
    return sum(list);
}

A few more examples:

In [ ]:
List<? extends Number> numbers = Arrays.asList(1, 2.0, 3);  // we can only get from this list, no put operation allowed (except for null)
numbers.add(5);  // Error

List<? super Integer> numbers2 = Arrays.asList(1, 2.0, 3); // we can only put into this list, no get operation (except for null)
sum(numbers2); // Error

### Subtyping
We discussed earlier that `List<Integer` is not a `List<Number>`. So what is the common ancestor for both? The below diagram illustrates the hierarchy:

<img src="images/subtyping.png" width=600 height=auto/>

Also

<img src="images/subtyping2.png" width=600 height=auto/>

So why is `List<? extends Number` above in hierarchy to `List<? extends Integer>`? Because the former can accept wider range of types (`Double`, `Float`, `Integer`, etc). Whereas the latter can only accept (`Integer` and its subtypes). Similarly, `List<? super Integer>` can accept `Integer`, `Number`, etc; whereas `List<? super Number>` can only accept `Number` and `Object`.

**Wildcard capture:** compiler being able to infer the type of wildcard.

Some cases which can result in error:

In [ ]:
/*
set defined as <E> set(int, E)
get defined as <E> get(int)
*/

void getAndSet(List<?> list) {
    list.set(0, list.get(0));    // error
}

// The above statement leads to compilation error because the compiler doesn't know that both the
// invocations of get and set refer to the same type. The compiler cannot be sure that the type
// of list hasn't changed between invocations. What if there was a statement list = Arrays.asList(true, false)

In [ ]:
void swapFirst(List<? extends Number> l1, List<? extends Number> l2) {
    Number temp = l1.get(0);
    l1.set(0, l2.get(0)); // error
    l2.set(0, temp);      // error
}

## Type Erasure
The Java compiler removes type parameter after compilation:

In [ ]:
// Generic class
public class Box<T> {
    private T t;

    public void set(T t) {
        this.t = t;
    }

    public T get() {
        return t;
    }
}

// Gets converted to:
public class Box {
    private Object t;

    public void set(Object t) {
        this.t = t;
    }

    public Object get() {
        return t;
    }
}

If the type parameter was bounded, it gets replaced with the first bound:

In [ ]:
public class Pair<T extends Number, U extends List<T> & Serializable> {
    private T first;
    private U second;

    public Pair(T t, U u) {
        first = t;
        second = u;
    }

    // ...
}

// Gets converted to:
public class Pair {
    private Number first;
    private List second;

    public Pair(Number t, List u) {
        first = t;
        second = u;
    }

    // ...
}

Type erasure also happens with generic methods.  

**Bridge methods:** to preserve polymorphism, Java compiler adds additional methods to the class as illustrated below:

In [ ]:
public class Node<T> {
    public T data;

    public Node(T data) { this.data = data; }

    public void setData(T data) {
        this.data = data;
    }
}

/* --- Node gets converted to: ---
public class Node {
    public Object data;

    public Node(Object data) { this.data = data; }

    public void setData(Object data) {
        this.data = data;
    }
}
*/

public class DoubleNode extends Node<Integer> {
    public DoubleNode(Integer data) { super(data); }

    public void setData(Integer data) {
        super.setData(data);
    }
}

/* --- DoubleNode gets converted to: ---
public class DoubleNode extends Node {
    public DoubleNode(Integer data) { super(data); }

    public void setData(Integer data) {  // This class no longer overrides the one in Node, signature is different
        super.setData(data);
    }

    public void setData(Object data) {
        setData((Integer) data);  // This additional class added by the compiler
    }
}
*/

**Unchecked Cast:**  warning in Java appears when you cast a value to a parameterized (generic) type without the compiler being able to fully verify the type at runtime, due to type erasure.

In [ ]:
List rawList = new ArrayList();   // raw type
rawList.add("hello");

// Unchecked cast warning here
List<String> list = (List<String>) rawList;
String s = list.get(0); // Unsafe but compiles

We got a warning here because the raw list could have contained integer, boolean, etc. The last line would have thus failed.

**Type Erasure and Reflection:** although the type parameter is erased from the type, it still remains in the bytecode. Therefore the generic information can be accessed at runtime using reflection.

In [ ]:
class Value { 
    public List<String> numbers;

    public Map<String, Integer> getMap() {
        return null;
    }
}

/*--- Field ---*/
Field field = Value.class.getDeclaredField("numbers");
Type genericType = field.getGenericType();

if(genericType instanceof ParameterizedType parameterizedType) {
    Type[] typeArguments = parameterizedType.getActualTypeArguments();
    for(Type typeArgument : typeArguments) {
        System.out.println(typeArgument);
    }
}

/*--- Method ---*/
Method method = Value.class.getMethod("getMap");
Type genericReturnType = method.getGenericReturnType();

if(genericReturnType instanceof ParameterizedType parameterizedType) {
    Type[] typeArguments = parameterizedType.getActualTypeArguments();
    for (Type typeArgument : typeArguments) {
        System.out.println("Return type generic: " + typeArgument.getTypeName());
    }
}

## Restrictions

Cannot create instance of type parameter:

In [ ]:
E e = new E(); // Error
`
// Workaround
public E instantiate(Class<E> clazz) throws InstantiationException, IllegalAccessException {
    E e = clazz.newInstance();
    return e;
}

Cannot use wildcards in class instance creation expression:

In [ ]:
List<?> list = new ArrayList<?>(); // Error
Map<String, ? extends Number> map = new HashMap<String, ? extends Number>(); // Error

List<List<?>> list2 = new ArrayList<List<?>>(); // This is fine

Generic method call if it includes type parameters, then the type parameter cannot be wildcard:

In [ ]:
Arrays.<?>asList(1,2,3); // Error

Arrays.<List<?>>asList(numbers, integers); // This is fine

Cannot extend generic class having wildcards:

In [ ]:
class MyList extends List<?> { } // Error

No static field with generic type parameter:

In [ ]:
private static E e; // Not allowed

As a result of type erasure combining `instanceof` with generic type is not allowed

In [ ]:
public void process(List<E> list) {
    if(list instanceof List<Integer>) { // Error
        // ...
    }
}

Or cast to parameterised type

In [ ]:
List<Integer> intList = new ArrayList<>();
List<String> strList = (List<String>) intList; // error

ArrayList<Integer> anotherIntList = (ArrayList<Integer>) intList; // this is fine

Generic array creation is not allowed

In [ ]:
T[] array = new T[5]; // error
List<E>[] s = new List<E>[2]; // error
List<String>[] s = new List<String>[2]; // error

Cannot catch exception of parameterised type:

In [ ]:
try {
    // ...
} catch (T e) { // Assuming we had <T extends Exception> type definition somewhere above
    // ...
}